## Goal
* Gather sunset and sunrise times from an array of dates and locations and store results
* edited to retrieve values for just KMSO


https://sunrise-sunset.org/api

In [2]:
# Import tools
from urllib.request import urlopen
from datetime import timedelta, date, datetime
from dateutil import tz
import json
import pandas as pd
import time
import os.path

## Build Date Array

In [23]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# Initialize date range start at 6/30 to compensate for UTC
start_date = date(2017, 6, 30)
end_date = date(2017, 11, 1)

dates = []

for single_date in daterange(start_date, end_date):
        dates.append(single_date.strftime("%Y-%m-%d"))

In [24]:
print(dates[:4])
print(dates[-4:])

['2017-06-30', '2017-07-01', '2017-07-02', '2017-07-03']
['2017-10-28', '2017-10-29', '2017-10-30', '2017-10-31']


## Load Location Dictionary

In [52]:
col_names = ['STID', 'LATITUDE', 'LONGITUDE']

In [65]:
kmso_dict = [{'STID': 'KMSO', 'LATITUDE': 46.8701, 'LONGITUDE': -113.9953}]

In [76]:
# make a new DataFrame to load into build_url_df
station_kmso = pd.DataFrame(kmso_dict)

## Build URLs

In [78]:
def build_url_df(stations, dates): 
    url_station = []
    # loop through stations in weather_monitoring-locations.csv
    for index, row in stations.iterrows():
        # loop through date range set above
        for date in dates:
            row_id = row['STID']
            lat = row['LATITUDE']
            lon = row['LONGITUDE']
            row_url = 'https://api.sunrise-sunset.org/json?lat=' + str(lat) + '&lng=' + str(lon) + '&date=' + date + '&formatted=0'
            url_station.append(dict({'station_id': row_id, 'url': row_url, 'date': date}))
    return(url_station)

In [80]:
# Generate url list for api calls
url_batch = build_url_df(station_kmso, dates)

In [81]:
print('Number of urls: ' + str(len(url_batch)))
print('----\n', url_batch[-1:])

Number of urls: 124
----
 [{'station_id': 'KMSO', 'url': 'https://api.sunrise-sunset.org/json?lat=46.8701&lng=-113.9953&date=2017-10-31&formatted=0', 'date': '2017-10-31'}]


## Call API and store results

In [71]:
def api_call(url):
    f = urlopen(url)
    json_string = f.read()
    parsed_json = json.loads(json_string)
    
    return parsed_json['results']

In [72]:
def save_results(df_day, filepath):
    if os.path.isfile(filepath):
        with open(filepath, 'a') as f:
            df_day.to_csv(f, header=False, index=False)
    else:
        df_day.to_csv(filepath, index=False)

In [73]:
%%time
successfull_calls = 0

# Loop over url_batch to call api and store results
for url in url_batch:
    # sleep to limit API call speed
    #time.sleep(3)
    
    print(url['station_id'] + '\n' + url['url'])
    result = api_call(url['url'])
    
    # include station_id
    result['station_id'] = url['station_id']
    
    # include date 
    result['date'] = url['date']
    
    df_day = pd.DataFrame.from_dict([result])

    save_results(df_day, '../data/sunrise_sunset-KMSO_190209.csv')
    
    successfull_calls += 1
    print('successfull_calls: ' + str(successfull_calls))

WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-06-30&formatted=0
successfull_calls: 1
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-01&formatted=0
successfull_calls: 2
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-02&formatted=0
successfull_calls: 3
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-03&formatted=0
successfull_calls: 4
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-04&formatted=0
successfull_calls: 5
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-05&formatted=0
successfull_calls: 6
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-06&formatted=0
successfull_calls: 7
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-07&f

successfull_calls: 64
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-02&formatted=0
successfull_calls: 65
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-03&formatted=0
successfull_calls: 66
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-04&formatted=0
successfull_calls: 67
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-05&formatted=0
successfull_calls: 68
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-06&formatted=0
successfull_calls: 69
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-07&formatted=0
successfull_calls: 70
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-08&formatted=0
successfull_calls: 71
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.258

successfull_calls: 127
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-03&formatted=0
successfull_calls: 128
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-04&formatted=0
successfull_calls: 129
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-05&formatted=0
successfull_calls: 130
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-06&formatted=0
successfull_calls: 131
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-07&formatted=0
successfull_calls: 132
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-08&formatted=0
successfull_calls: 133
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-09&formatted=0
successfull_calls: 134
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=

KeyboardInterrupt: 

#### Additional Resources
[USNO](http://aa.usno.navy.mil/data/docs/RS_OneYear.php)